# Data Integration with CHAKRA

## Import CHAKRA library

In [1]:
using Chakra

┌ Info: Precompiling Chakra [23337c43-347c-4169-b193-f31a84c326bf]
└ @ Base loading.jl:1423


## Implement an interface for a data source S1

The module `S1` is CHAKRA data source. `Chakra.@Reference Int []` is a macro which loads the reference implementation of the Chakra interface. It defines three types, `Id`, `Constituent` and `Hierarchy`, and extends the generic Chakra interface functions with methods specific to these types. The first argument `Int` is the type used to encode ids. The second argument is a list of data source modules which will be encapsulated by `S1`. `@DefineAtt(:A,Int)` is a macro which defines an attribute named `:A` with type `Int`. `data` is the knowledge structure made available by the module. It is constructed using the constructor operations from the interface.

In [2]:
module S1

using Main.Chakra

Chakra.@Reference Int

@DefineAttribute :A Int

data = ins(Id(1),seta(:A,10,agg(Constituent)),emp(Hierarchy))

end

Main.S1

The type of `S1.data` is `Main.S1.Hierarchy`. The type of Chakra data should be unique for a given data source.

In [3]:
typeof(S1.data)

Main.S1.Hierarchy

We can use the generic Chakra functions to interface with the data. For example, `dom` takes a hierarchy and returns the list of identifiers which it defines.

In [4]:
dom(S1.data)

1-element Vector{Main.S1.Id}:
 Main.S1.Id(1)

Similarly, `cts` takes a hierarchy and returns a list of Id-Constituent pairs. 

In [5]:
cts(S1.data)

1-element Vector{Pair{Main.S1.Id, Main.S1.Constituent}}:
 Main.S1.Id(1) => Main.S1.Constituent(Dict{Symbol, Any}(:A => 10), Dict{Symbol, Any}(), Main.S1.Id[])

`fnd` takes and id and a hierarchy and returns the constituent bound to that id. 

In [6]:
c1 = fnd(S1.Id(1),S1.data)

Main.S1.Constituent(Dict{Symbol, Any}(:A => 10), Dict{Symbol, Any}(), Main.S1.Id[])

In [ ]:
c1.attributes

If an identifier is not present in a hierarchy, `fnd` will return None. 

In [ ]:
fnd(S1.Id(2),S1.data)

We can also extract attribute values from constituents. The attributes defined on a Chakra representation can be listed: (TODO: implement a proper function for this.)

In [ ]:
methods(S1.__attributes__)

`geta` takes a symbol representing an attribute name and a constituent and returns the value 

In [ ]:
geta(:A,c1)

`geta` can also be applied to an id and a hierarchy. This is defined as fnd(x) >=> geta(a). 

In [ ]:
geta("Main.S1.A",S1.Id(1),S1.data)

If a constituent has no value for an attribute, or if the constituent does not exist, `geta` will return None.

In [ ]:
geta("Main.S1.A",S1.Id(2),S1.data)

Trying to get attributes which have not been defined throws an error. 

In [ ]:
geta(:B,c1)

In [ ]:
geta("Main.S1.B",c1)

## Implement a second data source interface S2

In [ ]:
module S2

using Main.Chakra

Chakra.@Reference String

struct B
    value::Int
end

@DefineAttribute(:B,B)

data = ins(Id("Two"),seta(:B,B(99),agg(Id)),emp(Hierarchy))

end

In [ ]:
dom(S2.data)

In [ ]:
c2 = fnd(S2.Id("Two"),S2.data)

In [ ]:
geta(:B,c2)

## Link data sources S1 and S2 using a third data source S3

Module S3 is a data source which which encapsulates S1 and S2.

In [ ]:
module S3

using Main.Chakra

using Main.S1, Main.S2

Chakra.@Reference Symbol [S1,S2]

@DefineProperty(:TYPE,String)

data = ins(Id(:a),setp(:TYPE,"BinaryAssociation",agg(ID_TYPES[S1.Id(1),S2.Id("Two")])),emp(Hierarchy))

end

Hierarchies built using the S3 constructors can be thought of as existing on top of the hierarchies defined in S1 and S2. As a result, calling on isemp on emp(S3.Hierarchy) does not return true as S1.data and S2.data are not empty. 

In [ ]:
isemp(emp(S3.Hierarchy))

S3 hierarchy destructors will dispatch to S1.data and S2.data. Calling cts and dom on S3 hierarchies returns not only the constituents defined in S3 but also those defined in S1.data and S2.data. Similarly, calling fnd on S3 hierarchies will dispatch according to the type of the identifier passed. 

In [ ]:
cts(S3.data)

In [ ]:
c3 = fnd(S3.Id(:a),S3.data)

In [ ]:
fnd(S1.Id(1),S3.data)

In [ ]:
fnd(S2.Id("Two"),S3.data)

Operations derived from the Chakra interface can be generically applied to data from S3, leaving dispatch to the compiler. 

In [ ]:
ps = pts(c3)

In [ ]:
s = sequence(ps,S3.data)

In [ ]:
sequence(S3.Id(:a),S3.data)

In [ ]:
x = gethead(s)

In [ ]:
geta(:A,x)

In [ ]:
Chakra.peek(S3.data)